In [1]:
from google.colab import drive
import zipfile
import os

# Mount Google Drive
drive.mount('/content/drive')

# Path to your ZIP file in Google Drive
zip_path = '/content/drive/MyDrive/dog-breed-identification.zip'

# Destination to extract the ZIP
extract_path = '/content/dog-breed-identification'

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Set the new dataset path for use in the rest of the code
dataset_path = extract_path


Mounted at /content/drive


In [2]:
import torch
import os
from PIL import Image
from pathlib import Path
import pandas as pd
import torchvision
from sklearn.model_selection import train_test_split

In [3]:
labels=pd.read_csv('/content/dog-breed-identification/labels.csv')
train,valid=train_test_split(labels,train_size=0.8,shuffle=True,stratify=labels['breed'],random_state=42)

In [4]:
train,train_labels=train['id'].reset_index(drop=True),train['breed'].reset_index(drop=True)
val,val_labels=valid['id'].reset_index(drop=True),valid['breed'].reset_index(drop=True)

In [5]:
'''Encoding labels'''

breeds=dict()
breed_count=1

for breed in labels['breed'].value_counts().index:

    breeds[breed]=breed_count-1
    breed_count+=1

val_labels_torch=torch.zeros(len(val_labels),1)
train_labels_torch=torch.zeros(len(train_labels),1)

for index in val_labels.index:
    val_labels_torch[index]=breeds[val_labels.iloc[index]]

for index in train_labels.index:
    train_labels_torch[index]=breeds[train_labels.iloc[index]]

val_labels_torch = val_labels_torch.long()
train_labels_torch = train_labels_torch.long()


val_labels_torch[91]
# val_labels=torch.tensor(val_labels.values)
# train_labels=torch.tensor(train_labels.values)

tensor([21])

In [6]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(10),
    torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    torchvision.transforms.ToTensor()
])


def create_dataset(series, directory_path, transform):
    tensors = []
    for i in range(len(series)):
        if(series[i].endswith('.jpg')):
            img_file=series[i]
        else:
            img_file = series[i] + '.jpg'
        img_path = os.path.join(directory_path, img_file)
        img = Image.open(img_path).convert('RGB')  # Ensure RGB
        img_t = transform(img)
        tensors.append(img_t)
    return torch.stack(tensors, dim=0)


In [7]:
directory_path=Path('/content/dog-breed-identification/train')
train_dataset=create_dataset(train,directory_path,transform)

In [8]:


# For validation: no augmentation
val_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor()
])
val_dataset = create_dataset(val, directory_path, transform=val_transform)


In [9]:
from torch.utils.data import TensorDataset

train_dataset_torch=TensorDataset(train_dataset,train_labels_torch)
val_dataset_torch=TensorDataset(val_dataset,val_labels_torch)

In [10]:
train_dataset.shape,train_labels_torch.shape

(torch.Size([8177, 3, 224, 224]), torch.Size([8177, 1]))

In [11]:
val_dataset_torch[0]

(tensor([[[0.2980, 0.2667, 0.1569,  ..., 0.2510, 0.0471, 0.0353],
          [0.2275, 0.1490, 0.2157,  ..., 0.1020, 0.0196, 0.0078],
          [0.2000, 0.1098, 0.2745,  ..., 0.0510, 0.0275, 0.0157],
          ...,
          [0.1176, 0.2118, 0.1843,  ..., 0.1176, 0.1373, 0.1451],
          [0.1490, 0.1843, 0.1804,  ..., 0.1765, 0.1569, 0.1451],
          [0.2431, 0.2784, 0.2118,  ..., 0.1529, 0.1216, 0.1059]],
 
         [[0.3765, 0.3373, 0.2275,  ..., 0.2863, 0.0549, 0.0314],
          [0.3137, 0.2314, 0.2980,  ..., 0.1373, 0.0275, 0.0039],
          [0.2941, 0.2039, 0.3647,  ..., 0.0824, 0.0392, 0.0157],
          ...,
          [0.1176, 0.2118, 0.1882,  ..., 0.1333, 0.1529, 0.1608],
          [0.1490, 0.1843, 0.1843,  ..., 0.1843, 0.1647, 0.1529],
          [0.2431, 0.2784, 0.2157,  ..., 0.1569, 0.1255, 0.1059]],
 
         [[0.3765, 0.3608, 0.2588,  ..., 0.2235, 0.0431, 0.0510],
          [0.2667, 0.2000, 0.2706,  ..., 0.0745, 0.0157, 0.0235],
          [0.1961, 0.1176, 0.2902,  ...,

In [12]:
from torchvision.models import resnet101, ResNet101_Weights
import torch.nn as nn
import torch.optim as optim

resnet = resnet101(weights=ResNet101_Weights.IMAGENET1K_V1)
for param in resnet.parameters():
    param.requires_grad = False


resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 256),  # From ResNet's output to 256 units
    nn.ReLU(),                              # Activation function
    nn.BatchNorm1d(256),                    # Normalization for stability
    nn.Linear(256, 256),                    # Additional dense layer
    nn.ReLU(),                              # Activation again
    nn.Dropout(0.3),                        # Dropout to reduce overfitting
    nn.BatchNorm1d(256),                    # Another batch norm layer
    nn.Linear(256, 120)                     # Final layer: 120 output classes
)
model_torch = resnet

optimizier=optim.Adam(model_torch.parameters(),lr=1e-4)
loss_fn=nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 125MB/s]


In [13]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset_torch, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset_torch, batch_size=64, shuffle=False)

n_epochs=10
model_torch=model_torch.to('cuda')
for i in range(1,n_epochs+1):
    model_torch.train()
    training_loss=0.0
    for img,label in train_loader:
        img=img.to('cuda')
        label=label.to('cuda')
        label=label.squeeze(1)
        outputs=model_torch(img)
        optimizier.zero_grad()
        losses=loss_fn(outputs,label)
        training_loss+=losses.item()
        losses.backward()
        optimizier.step()

    model_torch.eval()
    val_loss=0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for img,label in val_loader:
            img=img.to('cuda')
            label=label.to('cuda')
            label=label.squeeze(1)
            outputs=model_torch(img)
            loss=loss_fn(outputs,label)
            val_loss+=loss.item()

            _, predicted = torch.max(outputs, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
    print(f"Epoch [{i}/{n_epochs}], Training Loss: {training_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")


Epoch [1/10], Training Loss: 3.9210, Validation Loss: 3.0308, Accuracy: 59.56%
Epoch [2/10], Training Loss: 2.6958, Validation Loss: 2.2428, Accuracy: 70.56%
Epoch [3/10], Training Loss: 2.0731, Validation Loss: 1.7893, Accuracy: 74.96%
Epoch [4/10], Training Loss: 1.6755, Validation Loss: 1.4953, Accuracy: 77.70%
Epoch [5/10], Training Loss: 1.4055, Validation Loss: 1.2872, Accuracy: 79.66%
Epoch [6/10], Training Loss: 1.2046, Validation Loss: 1.1519, Accuracy: 80.15%
Epoch [7/10], Training Loss: 1.0582, Validation Loss: 1.0529, Accuracy: 80.88%
Epoch [8/10], Training Loss: 0.9267, Validation Loss: 0.9677, Accuracy: 81.96%
Epoch [9/10], Training Loss: 0.8303, Validation Loss: 0.9054, Accuracy: 80.88%
Epoch [10/10], Training Loss: 0.7427, Validation Loss: 0.8663, Accuracy: 81.32%


In [16]:
# ——— Save model + class mapping & print class names ———

import torch

# 1) Attach your breeds dict to the model
#    (make sure you ran the cell that built `breeds = {...}` from your CSV)
# Changed 'model' to 'model_torch' to match the variable name used for the model
model_torch.class_to_idx = breeds

# 2) Build checkpoint with both weights and mapping
checkpoint = {
    # Changed 'model.state_dict()' to 'model_torch.state_dict()'
    'model_state_dict': model_torch.state_dict(),
    'class_to_idx':       model_torch.class_to_idx,
    # optionally: 'optimizer_state_dict': optimizer.state_dict(),
    #              'epoch': epoch,
}

# 3) Save it
torch.save(checkpoint, 'dog_breed_model_with_names.pth')
print("✅ Saved model + class_to_idx mapping to dog_breed_model_with_names.pth\n")

# 4) Verify by inverting and printing every index → breed name
# Changed 'model.class_to_idx' to 'model_torch.class_to_idx'
idx_to_class = {idx: breed for breed, idx in model_torch.class_to_idx.items()}
print(f"Found {len(idx_to_class)} classes:")
for idx in sorted(idx_to_class):
    print(f"{idx:3d} → {idx_to_class[idx]}")

✅ Saved model + class_to_idx mapping to dog_breed_model_with_names.pth

Found 120 classes:
  0 → scottish_deerhound
  1 → maltese_dog
  2 → afghan_hound
  3 → entlebucher
  4 → bernese_mountain_dog
  5 → shih-tzu
  6 → great_pyrenees
  7 → pomeranian
  8 → basenji
  9 → samoyed
 10 → tibetan_terrier
 11 → airedale
 12 → leonberg
 13 → cairn
 14 → japanese_spaniel
 15 → beagle
 16 → australian_terrier
 17 → miniature_pinscher
 18 → blenheim_spaniel
 19 → irish_wolfhound
 20 → saluki
 21 → lakeland_terrier
 22 → papillon
 23 → norwegian_elkhound
 24 → whippet
 25 → siberian_husky
 26 → pug
 27 → chow
 28 → italian_greyhound
 29 → pembroke
 30 → ibizan_hound
 31 → border_terrier
 32 → newfoundland
 33 → lhasa
 34 → silky_terrier
 35 → dandie_dinmont
 36 → bedlington_terrier
 37 → sealyham_terrier
 38 → rhodesian_ridgeback
 39 → irish_setter
 40 → old_english_sheepdog
 41 → collie
 42 → boston_bull
 43 → schipperke
 44 → kelpie
 45 → african_hunting_dog
 46 → bouvier_des_flandres
 47 → eng

In [18]:
# ====== Load model + CHECK class names ======
import torch
from torchvision import models  # or wherever your architecture lives
import torch.nn as nn # Import nn for Sequential and Linear

# 1. Re-instantiate your model architecture EXACTLY as used for training.
#    This means using resnet101 and the custom fc layer structure.
# Use resnet101 without pre-trained weights, as you will load your trained weights
model = models.resnet101(pretrained=False)
num_classes = 120

# Recreate the same Sequential layer for the final fully connected layer
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),  # From ResNet's output to 256 units
    nn.ReLU(),                              # Activation function
    nn.BatchNorm1d(256),                    # Normalization for stability
    nn.Linear(256, 256),                    # Additional dense layer
    nn.ReLU(),                              # Activation again
    nn.Dropout(0.3),                        # Dropout to reduce overfitting
    nn.BatchNorm1d(256),                    # Another batch norm layer
    nn.Linear(256, num_classes)             # Final layer: 120 output classes
)


# 2. Load the checkpoint
checkpoint = torch.load('/content/dog_breed_model_with_names.pth', map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])

# 3. Recover the mapping dicts
if 'class_to_idx' in checkpoint:
    class_to_idx = checkpoint['class_to_idx']
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    print(f"Found {len(idx_to_class)} classes:\n")
    # 4. Print them in index order
    for idx in sorted(idx_to_class):
        print(f"{idx:3d} → {idx_to_class[idx]}")
else:
    print("No class_to_idx found in checkpoint.")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Found 120 classes:

  0 → scottish_deerhound
  1 → maltese_dog
  2 → afghan_hound
  3 → entlebucher
  4 → bernese_mountain_dog
  5 → shih-tzu
  6 → great_pyrenees
  7 → pomeranian
  8 → basenji
  9 → samoyed
 10 → tibetan_terrier
 11 → airedale
 12 → leonberg
 13 → cairn
 14 → japanese_spaniel
 15 → beagle
 16 → australian_terrier
 17 → miniature_pinscher
 18 → blenheim_spaniel
 19 → irish_wolfhound
 20 → saluki
 21 → lakeland_terrier
 22 → papillon
 23 → norwegian_elkhound
 24 → whippet
 25 → siberian_husky
 26 → pug
 27 → chow
 28 → italian_greyhound
 29 → pembroke
 30 → ibizan_hound
 31 → border_terrier
 32 → newfoundland
 33 → lhasa
 34 → silky_terrier
 35 → dandie_dinmont
 36 → bedlington_terrier
 37 → sealyham_terrier
 38 → rhodesian_ridgeback
 39 → irish_setter
 40 → old_english_sheepdog
 41 → collie
 42 → boston_bull
 43 → schipperke
 44 → kelpie
 45 → african_hunting_dog
 46 → bouvier_des_flandres
 47 → english_foxhound
 48 → weimaraner
 49 → bloodhound
 50 → bluetick
 51 → la

In [19]:
!cp /content/dog_breed_model_with_names.pth /content/drive/MyDrive/
